## gemini ground truth gemini output in json 

In [ ]:
# to convert docx.pdf to pdf
import os

root_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr"

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.docx.pdf'):
            old_path = os.path.join(dirpath, filename)
            new_filename = filename[:-9] + '.pdf'  # Remove 9 chars: '.docx.pdf'
            new_path = os.path.join(dirpath, new_filename)
            if os.path.exists(new_path):
                print(f"Could not rename (target exists): {old_path}")
            else:
                try:
                    os.rename(old_path, new_path)
                except Exception as e:
                    print(f"Could not rename (error): {old_path} - {e}")


Found file: 01_10021040891039611111693747018.docx.pdf
Trying to rename: /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr/01_10021040891039611111693747018.docx.pdf -> /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr/01_10021040891039611111693747018.pdf
Renamed: /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr/01_10021040891039611111693747018.docx.pdf -> /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr/01_10021040891039611111693747018.pdf
Found file: 08_1002109149818371111684649303.docx.pdf
Trying to rename: /Users/simra

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# === Load API Key ===
load_dotenv()

model_name = "gemini-2.5-pro"
genai.configure(api_key=os.getenv("GOOGLE_GEMINI_API"))
model = genai.GenerativeModel(model_name)

def send_pdf_to_gemini_and_save_json(pdf_path, prompt, output_base_dir):
    try:
        # Upload the PDF file
        file_resource = genai.upload_file(pdf_path, mime_type="application/pdf")
        
        # Compose the prompt and file
        response = model.generate_content([prompt, file_resource])
        generated_text = response.text

        # Extract the filename and set output directory for JSON file
        pdf_filename = os.path.basename(pdf_path)
        pdf_stem = os.path.splitext(pdf_filename)[0]
        output_dir = os.path.join(output_base_dir, pdf_stem)
        os.makedirs(output_dir, exist_ok=True)

        # Define the output JSON path
        output_json_path = os.path.join(output_dir, f"{pdf_stem}.json")

        # Write the output JSON file
        with open(output_json_path, 'w', encoding='utf-8') as output_file:
            output_file.write(generated_text)
        
        print(f"🎉 Output saved to {output_json_path}")
    except Exception as e:
        print(f"❌ Error in generating response for {pdf_path}: {str(e)}")

# Function to process all PDF files in a directory
def process_all_pdfs(input_dir, output_dir, prompt):
    # Iterate through all files in the directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".pdf"):
                input_pdf_path = os.path.join(root, file)
                print(f"Processing {input_pdf_path}...")
                send_pdf_to_gemini_and_save_json(input_pdf_path, prompt, output_dir)

# === Example Usage ===
if __name__ == "__main__":
    # Input directory containing PDF files
    input_pdf_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr"  # Change this to the correct directory path
    output_json_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/pdf_docx_json"  # Change this to the desired output directory
    prompt = """
*Role**: You are a meticulous digital archivist tasked with transcribing handwritten student answer sheets.

**Core Task**: Your goal is to create a perfect digital copy of the student's work. You must transcribe the text *exactly* as it appears, including any spelling or grammatical errors.
---

### Other Directives
1.  **Collate Sub-Questions**: Group all parts of a question (e.g., 11. (1), 11. (2)) under a single main question number. Preserve the original sub-question numbering in the text.

---

### Output Format
- The output MUST be a single, valid JSON array containing one object per main question.
- Do NOT include any text or explanations outside of the JSON array.
- Only transcribe text that is not enclosed within angle brackets (<TEXT>).

**Example of a valid JSON object:**
[
  {
    "question_number": 11,
    "ocr_text": "11. (1) This is the answer to the first part. 11. (2) This is the final answer, which contains a PV curve. [NOTE: The student originally wrote 'the initial answer' here and struck it through; it has been correctly omitted from this output per the critical rule.]",
    "diagrams": [
      {
        "id": "diagram_1",
        "coordinates": "0.5,0.5,0.2,0.3",
        "diagram_class": "graph or diagram",
        "page_number": 3
      }
    ],
    "pages": [2, 3]
  }
]

**Schema Definitions:**
- question_number (integer): The main question number.
- ocr_text (string): The full, collated text for the question and all its sub-parts.
- diagrams (array): A list of diagram objects. Leave as an empty array `[]` if none.
  - id (string): The diagram identifier from the text.
  - diagram_class: mention is it a diagram or a graph or a number line .
  - coordinates (string): "x_mid,y_mid,width,height", with values normalized between 0 and 1 relative to image dimensions.
  - page_number (integer): The page where the diagram is located.
- pages (array): A list of all page numbers on which any part of the question appears.
"""
    # Start processing all PDFs in the input directory
    process_all_pdfs(input_pdf_dir, output_json_dir, prompt)


Processing /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/Physics_pdf_docx_human_ocr/12_1002140198994121111692513661.pdf...


## to send pdf to gemini get ocr output for prediction

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import fitz  # PyMuPDF for PDF processing
from PIL import Image
import json
import base64
import cv2
import time
import numpy as np

# === Load API Key ===
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_GEMINI_API"))

# Set model
model_name = "gemini-2.5-pro"
model = genai.GenerativeModel(model_name)

# === Prompt for Gemini ===
PROMPT = """
### System Instruction

**Role**: You are a meticulous digital archivist tasked with transcribing handwritten student answer sheets.

**Core Task**: Your goal is to create a perfect digital copy of the student's work.

**Important Rule**: **DO NOT** correct any spelling, punctuation, or grammatical errors. This includes cases where words might seem misspelled, such as "metabolities" instead of "metabolites". **Preserve all text exactly as it appears in the PDF**, even if there are apparent mistakes or inconsistencies.
- Please avoid using Unicode escape sequences (e.g., \u25b3) and provide direct characters like ∆, ², and cm.
- Whenever a table is found in the image, output the information in a proper table format.
- Please ignore symbols like  "->" from the final text output.
- If the word "Ans." is followed by a number (e.g., "Ans1."), only include the number and exclude the word "Ans."

---

### IGNORE ALL STRIKETHROUGH TEXT
Any portion of text that has a line through it (strikethrough) MUST BE COMPLETELY OMITTED from the output. Do not include it, do not describe it, and do not reference it in any way. Treat it as if it does not exist.

---

### Other Directives
1.  **Ignore Page Template**: Exclude all non-content elements like headers, footers, page numbers, or decorative logos.
2.  **Collate Sub-Questions**: Group all parts of a question (e.g., 11. (1), 11. (2)) under a single main question number. Preserve the original sub-question numbering in the text.

---

### Output Format
- The output MUST be a single, valid JSON array containing one object per main question.
- Do NOT include any text or explanations outside of the JSON array.

**Example of a valid JSON object:**
[
  {
    "question_number": 11,
    "ocr_text": "11. (1) This is the answer to the first part. 11. (2) This is the final answer, which contains a PV curve. [NOTE: The student originally wrote 'the initial answer' here and struck it through; it has been correctly omitted from this output per the critical rule.]",
    "diagrams": [
      {
        "id": "diagram_1",
        "coordinates": "0.5,0.5,0.2,0.3",
        "diagram_class": "graph or diagram",
        "page_number": 3
      }
    ],
    "pages": [2, 3]
  }
]


**Schema Definitions:**
- `question_number` (integer): The main question number.
- `ocr_text` (string): The full, collated text for the question and all its sub-parts.
- `diagrams` (array): A list of diagram objects. Leave as an empty array `[]` if none.
  - `id` (string): The diagram identifier from the text.
  - `coordinates` (string): "x_mid,y_mid,width,height", with values normalized between 0 and 1 relative to image dimensions.
  - `page_number` (integer): The page where the diagram is located.
  - diagram_class: mention is it a diagram or a graph .
- `pages` (array): A list of all page numbers on which any part of the question appears.
"""

# === Set the dimension value ===
dim = 1536  # Define the dimension value

# === Resize Image Function ===
def resize_image(image, dim=dim, save_path=None):
    image1 = np.array(image.convert('RGB'))  # Ensure the image is in RGB mode
    original_size = image1.shape  # (height, width, channels)
    image1 = image1.mean(axis=2)  # Convert image to grayscale
    h, w = image1.shape
    if w > h:
        new_w = dim
        new_h = int(h * (dim / w))
    else:
        new_h = dim
        new_w = int(w * (dim / h))
    resized_image = cv2.resize(image1, (new_w, new_h), interpolation=cv2.INTER_AREA)
    resized_image_pil = Image.fromarray(resized_image)
    resized_image_pil = resized_image_pil.convert('RGB')  # Convert to RGB before saving
    if save_path:
        resized_image_pil.save(save_path)
    return original_size, (new_h, new_w), resized_image_pil

# === Load PDF, Convert Pages to Images ===
def pdf_to_images(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    images = []
    num_pages = doc.page_count
    for i in range(num_pages):
        page = doc.load_page(i)
        pix = page.get_pixmap(dpi=300)
        img_path_default = os.path.join(output_folder, f"page_{i + 1}.jpeg")
        pix.save(img_path_default)
        images.append(img_path_default)
    return images, num_pages

# === Load Base64 Images ===
def load_base64_images(folder_path, num_pages):
    b64_list = []
    for i in range(num_pages):
        img_filename_dim = f"DIM_{dim}_PAGE_{i + 1}.jpeg"  # Use dim variable here
        img_filename_default = f"page_{i + 1}.jpeg"  # Old naming convention

        img_path_dim = os.path.join(folder_path, img_filename_dim)
        img_path_default = os.path.join(folder_path, img_filename_default)

        if os.path.exists(img_path_dim):
            path = img_path_dim
        elif os.path.exists(img_path_default):
            path = img_path_default
        else:
            print(f"❌ Image {img_filename_dim} or {img_filename_default} not found in {folder_path}")
            continue

        with open(path, "rb") as f:
            b64_list.append(base64.b64encode(f.read()).decode())
    return b64_list

# === Batch send to Gemini ===
def send_to_gemini(resized_images_objs):
    try:
        response = model.generate_content([PROMPT] + resized_images_objs)  # Batch processing
        raw = response.text.strip()
        cleaned = raw.strip('```json').strip('```').strip()
        parsed = json.loads(cleaned)
        return parsed
    except Exception as e:
        print(f"❌ Failed to process images: {e}")
        return None

# === Function to process all PDF files in a directory ===
def process_all_pdfs(input_pdf_dir, output_json_dir):
    # Iterate through all files in the directory
    for root, dirs, files in os.walk(input_pdf_dir):
        for file in files:
            if file.endswith(".pdf"):
                input_pdf_path = os.path.join(root, file)
                print(f"Processing {input_pdf_path}...")  # Log the current file being processed
                # Define the output folder path using the PDF filename
                pdf_filename = os.path.splitext(file)[0]
                output_folder = os.path.join(output_json_dir, pdf_filename)
                os.makedirs(output_folder, exist_ok=True)

                # Process each PDF file
                try:
                    # Step 1: Convert PDF to images
                    images, num_pages = pdf_to_images(input_pdf_path, output_folder)

                    # Step 2: Resize images
                    resized_images = []
                    for page_num in range(len(images)):
                        img_path = images[page_num]
                        original_size, new_size, resized_image = resize_image(Image.open(img_path), dim=dim)
                        
                        # Save resized image
                        img_filename_dim = f"DIM_{dim}_PAGE_{page_num + 1}.jpeg"
                        resized_image.save(os.path.join(output_folder, img_filename_dim))
                        
                        resized_images.append(resized_image)

                    # Step 3: Load base64 images
                    images_b64 = load_base64_images(output_folder, len(images))

                    # Step 4: Send to Gemini for OCR
                    results = send_to_gemini(resized_images)

                    if results:
                        output_json_filename = f"{pdf_filename}.json"
                        json_path = os.path.join(output_folder, output_json_filename)

                        # Save the OCR results
                        with open(json_path, "w") as f:
                            json.dump(results, f, indent=3)

                        print(f"OCR results saved to {json_path}")
                    else:
                        print("❌ No results from Gemini OCR")

                except Exception as e:
                    print(f"❌ Error processing {input_pdf_path}: {e}")


# === Example Usage ===
if __name__ == "__main__":
    input_pdf_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/phy"  # Change this to the correct directory path
    output_json_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json"  # Change this to the desired output directory
    
    # Process all PDFs in the input directory
    process_all_pdfs(input_pdf_dir, output_json_dir)


Processing /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Maths_human/maths_pdf_docx_human_ocr/11_10021019941080491171694789012.pdf...


KeyboardInterrupt: 

## Make a table with human ocr , gemini ocr , cer , type of error ,  error discription , question number, docx
"""
Spelling

Wording

Extra Content

Punctuation

Numerical Difference

Missing Content

Content Mix-up

Omission

"""

In [5]:
import os
import json
from difflib import ndiff

GEMINI_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json"
HUMAN_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/pdf_docx_json"
OUTPUT_MD_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables"

os.makedirs(OUTPUT_MD_DIR, exist_ok=True)

def char_error_rate(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    insertions = sum(1 for d in diff if d[0] == '+')
    deletions = sum(1 for d in diff if d[0] == '-')
    ref_len = len(s1)
    if ref_len == 0:
        return 0 if len(s2) == 0 else 1
    cer = (insertions + deletions) / ref_len
    return cer

def highlight_differences(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    result = []
    for d in diff:
        if d[0] == ' ':
            result.append(d[2])
        elif d[0] == '-':
            result.append(f"[-{d[2]}-]")
        elif d[0] == '+':
            result.append(f"[+{d[2]}+]")
    return ''.join(result)

def sanitize(text):
    if text == "na":
        return text
    text = text.replace("\n", " ").replace("|", "\\|").replace(",", "&#44;")
    return text

def build_q_map(data):
    q_map = {}
    for entry in data:
        qnum = entry.get("question_number")
        text = entry.get("ocr_text", "")
        q_map[qnum] = text
    return q_map

header = [
    "docx source",
    "question number",
    "Human_ocr",
    "Gemini_ocr",
    "cer",
    "highlighted differences",
    "type of error",
    "Discrepancy Analysis"
]

for root, dirs, files in os.walk(GEMINI_JSON_DIR):
    for file in files:
        if not file.endswith(".json"):
            continue
        gemini_json_path = os.path.join(root, file)
        rel_path = os.path.relpath(gemini_json_path, GEMINI_JSON_DIR)
        human_json_path = os.path.join(HUMAN_JSON_DIR, rel_path)
        if not os.path.exists(human_json_path):
            print(f"❌ No matching human JSON for {gemini_json_path}")
            continue

        # Robust loading for Gemini JSON
        try:
            with open(gemini_json_path, "r", encoding="utf-8") as f:
                gemini_content = f.read()
                if not gemini_content.strip():
                    print(f"⚠️ Empty Gemini JSON: {gemini_json_path}")
                    continue
                gemini_data = json.loads(gemini_content)
        except Exception as e:
            print(f"❌ Error loading Gemini JSON: {gemini_json_path}\n{e}")
            continue

        # Robust loading for Human JSON
        try:
            with open(human_json_path, "r", encoding="utf-8") as f:
                human_content = f.read()
                if not human_content.strip():
                    print(f"⚠️ Empty Human JSON: {human_json_path}")
                    continue
                human_data = json.loads(human_content)
        except Exception as e:
            print(f"❌ Error loading Human JSON: {human_json_path}\n{e}")
            continue

        gemini_q = build_q_map(gemini_data)
        human_q = build_q_map(human_data)
        md_lines = ["# OCR Comparison Table", "\n", "| " + " | ".join(header) + " |", "| " + " | ".join(['---'] * len(header)) + " |"]
        cer_values = []
        for qnum in sorted(set(gemini_q.keys()) | set(human_q.keys())):
            human_text = sanitize(human_q.get(qnum, "na"))
            gemini_text = sanitize(gemini_q.get(qnum, "na"))
            if human_text == "na" or gemini_text == "na":
                cer_value = "na"
                highlighted_diff = "na"
            else:
                cer_value = char_error_rate(human_text, gemini_text)
                cer_values.append(cer_value)
                cer_value = f"{cer_value:.4f}"
                highlighted_diff = sanitize(highlight_differences(human_text, gemini_text))
            row = [
                file,
                str(qnum),
                human_text,
                gemini_text,
                cer_value,
                highlighted_diff,
                "",
                "",
            ]
            md_lines.append("| " + " | ".join(row) + " |")
        out_dir = os.path.join(OUTPUT_MD_DIR, os.path.dirname(rel_path))
        os.makedirs(out_dir, exist_ok=True)
        output_md_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.md")
        with open(output_md_path, "w", encoding="utf-8") as f:
            f.write("\n".join(md_lines))
        print(f"Markdown table written to {output_md_path}")
        # Write CER txt
        if cer_values:
            overall_cer = sum(cer_values) / len(cer_values)
            cer_txt_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.txt")
            with open(cer_txt_path, "w", encoding="utf-8") as f:
                f.write(f"Overall CER: {overall_cer:.4f}\n")
            print(f"CER written to {cer_txt_path}")
        else:
            print(f"No valid CER values for {file}")

Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables/12_1002140198994121111692513661/12_1002140198994121111692513661.md
CER written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables/12_1002140198994121111692513661/12_1002140198994121111692513661.txt
Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables/09_1002114885961841111690700733/09_1002114885961841111690700733.md
CER written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables/09_1002114885961841111690700733/09_1002114885961841111690700733.txt
Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/soluti

In [ ]:
import os

md_path = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/maths_final.md"

def parse_md_table(md_lines):
    table = []
    for line in md_lines:
        line = line.strip()
        if line.startswith('|') and line.endswith('|'):
            cells = [cell.strip() for cell in line[1:-1].split('|')]
            table.append(cells)
    return table

with open(md_path, 'r') as f:
    lines = f.readlines()

table = parse_md_table(lines)

if len(table) < 2:
    print("Table too short or missing header.")
    exit()

header = table[0]
try:
    cer_idx = [h.lower().replace(' ', '') for h in header].index('cer')
except ValueError:
    print("No 'cer' column found in header.")
    exit()

cer_values = []
zero_cer_count = 0
for row in table[2:]:  # skip header and separator
    if cer_idx < len(row):
        val = row[cer_idx].strip().lower()
        if val != 'na' and val != '':
            try:
                cer_float = float(val)
                cer_values.append(cer_float)
                if cer_float == 0:
                    zero_cer_count += 1
            except ValueError:
                print(f"Non-numeric CER value ignored: {row[cer_idx]}")

if cer_values:
    avg_cer = sum(cer_values) / len(cer_values)
    print(f"Average CER (ignoring 'na'): {avg_cer}")
    print(f"Number of valid CER rows: {len(cer_values)}")
    print(f"Number of rows where CER is 0: {zero_cer_count}")
else:
    print("No valid CER values found.")

Non-numeric CER value ignored: BC AP/PB = AD/DC (B.P.T) → (1) Ad/DC = CE/BE (Given) → (2) AP/PB = CF/BE (From (1) & (2)) by converse of BPT DE//AC
Non-numeric CER value ignored: DE To prove AD/DB = AE/EC Join BE&#44; CD Construction : EF⊥AB &#44; DG⊥CA ∴ Area of AADE = 1/2 x base x haight = 1/2 x AD x EF = 1/2 x AE x DG Area of ΔPBE = Area of ΔADE / Area of ΔDBE = AD/DB → (1) Similarly Area of ΔADE = 1/2 x AE x DG Area of ΔDCE = 1/2 x EC x DG = AE/EC ΔDBE and ΔDCE have the same base b'w same parallels ∴ AD/DB = AC/EC
Non-numeric CER value ignored: gm bisect each other. So the midpoint of AC is same of mid point of BD AC BD (x₁+x₂/2&#44; y₁+y₂/2) = (3+x/2&#44; 6+y/2) (3+x/2&#44; 6+y/2) = (3+x/2&#44; 6+y/2) (6/2&#44; 8/2) = (3+x/2&#44; 6+y/2) 3 + x = 6 ; 6 + y = 3 x = 3 ; y = 2 So forth vertex's coordinates are (3&#44;2)
Non-numeric CER value ignored: \
Non-numeric CER value ignored: BC Given → AD/CD = AP/BP [BPT -(1)] Also given → AD/CD = CE/BE -(2) From (1) & (2) AD/CD = AP/BP = CE/BE 

In [ ]:
import os
import json
from difflib import ndiff

GEMINI_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json"
HUMAN_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_human/pdf_docx_json"
OUTPUT_MD_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables"

os.makedirs(OUTPUT_MD_DIR, exist_ok=True)

def char_error_rate(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    insertions = sum(1 for d in diff if d[0] == '+')
    deletions = sum(1 for d in diff if d[0] == '-')
    ref_len = len(s1)
    if ref_len == 0:
        return 0 if len(s2) == 0 else 1
    cer = (insertions + deletions) / ref_len
    return cer

def highlight_differences(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    result = []
    for d in diff:
        if d[0] == ' ':
            result.append(d[2])
        elif d[0] == '-':
            result.append(f"[-{d[2]}-]")
        elif d[0] == '+':
            result.append(f"[+{d[2]}+]")
    return ''.join(result)

def sanitize(text):
    if text == "na":
        return text
    text = text.replace("\n", " ").replace("|", "\\|").replace(",", "&#44;")
    return text

def build_q_map(data):
    q_map = {}
    for entry in data:
        qnum = entry.get("question_number")
        text = entry.get("ocr_text", "")
        q_map[qnum] = text
    return q_map

header = [
    "docx source",
    "question number",
    "Human_ocr",
    "Gemini_ocr",
    "cer",
    "highlighted differences",
    "type of error",
    "Discrepancy Analysis"
]

for root, dirs, files in os.walk(GEMINI_JSON_DIR):
    for file in files:
        if not file.endswith(".json"):
            continue
        gemini_json_path = os.path.join(root, file)
        rel_path = os.path.relpath(gemini_json_path, GEMINI_JSON_DIR)
        human_json_path = os.path.join(HUMAN_JSON_DIR, rel_path)
        if not os.path.exists(human_json_path):
            print(f"❌ No matching human JSON for {gemini_json_path}")
            continue

        # Robust loading for Gemini JSON
        try:
            with open(gemini_json_path, "r", encoding="utf-8") as f:
                gemini_content = f.read()
                if not gemini_content.strip():
                    print(f"⚠️ Empty Gemini JSON: {gemini_json_path}")
                    continue
                gemini_data = json.loads(gemini_content)
        except Exception as e:
            print(f"❌ Error loading Gemini JSON: {gemini_json_path}\n{e}")
            continue

        # Robust loading for Human JSON
        try:
            with open(human_json_path, "r", encoding="utf-8") as f:
                human_content = f.read()
                if not human_content.strip():
                    print(f"⚠️ Empty Human JSON: {human_json_path}")
                    continue
                human_data = json.loads(human_content)
        except Exception as e:
            print(f"❌ Error loading Human JSON: {human_json_path}\n{e}")
            continue

        gemini_q = build_q_map(gemini_data)
        human_q = build_q_map(human_data)
        md_lines = ["# OCR Comparison Table", "\n", "| " + " | ".join(header) + " |", "| " + " | ".join(['---'] * len(header)) + " |"]
        cer_values = []
        for qnum in sorted(set(gemini_q.keys()) | set(human_q.keys())):
            human_text = sanitize(human_q.get(qnum, "na"))
            gemini_text = sanitize(gemini_q.get(qnum, "na"))
            if human_text == "na" or gemini_text == "na":
                cer_value = "na"
                highlighted_diff = "na"
            else:
                cer_value = char_error_rate(human_text, gemini_text)
                cer_values.append(cer_value)
                cer_value = f"{cer_value:.4f}"
                highlighted_diff = sanitize(highlight_differences(human_text, gemini_text))
            row = [
                file,
                str(qnum),
                human_text,
                gemini_text,
                cer_value,
                highlighted_diff,
                "",
                "",
            ]
            md_lines.append("| " + " | ".join(row) + " |")
        out_dir = os.path.join(OUTPUT_MD_DIR, os.path.dirname(rel_path))
        os.makedirs(out_dir, exist_ok=True)
        output_md_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.md")
        with open(output_md_path, "w", encoding="utf-8") as f:
            f.write("\n".join(md_lines))
        print(f"Markdown table written to {output_md_path}")
        # Write CER txt
        if cer_values:
            overall_cer = sum(cer_values) / len(cer_values)
            cer_txt_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.txt")
            with open(cer_txt_path, "w", encoding="utf-8") as f:
                f.write(f"Overall CER: {overall_cer:.4f}\n")
            print(f"CER written to {cer_txt_path}")
        else:
            print(f"No valid CER values for {file}")

In [ ]:
import os

md_path = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/maths_final.md"

def parse_md_table(md_lines):
    table = []
    for line in md_lines:
        line = line.strip()
        if line.startswith('|') and line.endswith('|'):
            cells = [cell.strip() for cell in line[1:-1].split('|')]
            table.append(cells)
    return table

with open(md_path, 'r') as f:
    lines = f.readlines()

table = parse_md_table(lines)

if len(table) < 2:
    print("Table too short or missing header.")
    exit()

header = table[0]
try:
    cer_idx = [h.lower().replace(' ', '') for h in header].index('cer')
except ValueError:
    print("No 'cer' column found in header.")
    exit()

cer_values = []
zero_cer_count = 0
for row in table[2:]:  # skip header and separator
    if cer_idx < len(row):
        val = row[cer_idx].strip().lower()
        if val != 'na' and val != '':
            try:
                cer_float = float(val)
                cer_values.append(cer_float)
                if cer_float == 0:
                    zero_cer_count += 1
            except ValueError:
                print(f"Non-numeric CER value ignored: {row[cer_idx]}")

if cer_values:
    avg_cer = sum(cer_values) / len(cer_values)
    print(f"Average CER (ignoring 'na'): {avg_cer}")
    print(f"Number of valid CER rows: {len(cer_values)}")
    print(f"Number of rows where CER is 0: {zero_cer_count}")
else:
    print("No valid CER values found.")

Non-numeric CER value ignored: BC AP/PB = AD/DC (B.P.T) → (1) Ad/DC = CE/BE (Given) → (2) AP/PB = CF/BE (From (1) & (2)) by converse of BPT DE//AC
Non-numeric CER value ignored: DE To prove AD/DB = AE/EC Join BE&#44; CD Construction : EF⊥AB &#44; DG⊥CA ∴ Area of AADE = 1/2 x base x haight = 1/2 x AD x EF = 1/2 x AE x DG Area of ΔPBE = Area of ΔADE / Area of ΔDBE = AD/DB → (1) Similarly Area of ΔADE = 1/2 x AE x DG Area of ΔDCE = 1/2 x EC x DG = AE/EC ΔDBE and ΔDCE have the same base b'w same parallels ∴ AD/DB = AC/EC
Non-numeric CER value ignored: gm bisect each other. So the midpoint of AC is same of mid point of BD AC BD (x₁+x₂/2&#44; y₁+y₂/2) = (3+x/2&#44; 6+y/2) (3+x/2&#44; 6+y/2) = (3+x/2&#44; 6+y/2) (6/2&#44; 8/2) = (3+x/2&#44; 6+y/2) 3 + x = 6 ; 6 + y = 3 x = 3 ; y = 2 So forth vertex's coordinates are (3&#44;2)
Non-numeric CER value ignored: \
Non-numeric CER value ignored: BC Given → AD/CD = AP/BP [BPT -(1)] Also given → AD/CD = CE/BE -(2) From (1) & (2) AD/CD = AP/BP = CE/BE 

In [3]:
import os
import json
from difflib import ndiff

GEMINI_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json"
HUMAN_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/pdf_docx_json"
OUTPUT_MD_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/tables"

os.makedirs(OUTPUT_MD_DIR, exist_ok=True)

def char_error_rate(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    insertions = sum(1 for d in diff if d[0] == '+')
    deletions = sum(1 for d in diff if d[0] == '-')
    ref_len = len(s1)
    if ref_len == 0:
        return 0 if len(s2) == 0 else 1
    cer = (insertions + deletions) / ref_len
    return cer

def highlight_differences(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    result = []
    for d in diff:
        if d[0] == ' ':
            result.append(d[2])
        elif d[0] == '-':
            result.append(f"[-{d[2]}-]")
        elif d[0] == '+':
            result.append(f"[+{d[2]}+]")
    return ''.join(result)

def sanitize(text):
    if text == "na":
        return text
    text = text.replace("\n", " ").replace("|", "\\|").replace(",", "&#44;")
    return text

def build_q_map(data):
    q_map = {}
    for entry in data:
        qnum = entry.get("question_number")
        text = entry.get("ocr_text", "")
        q_map[qnum] = text
    return q_map

header = [
    "docx source",
    "question number",
    "Human_ocr",
    "Gemini_ocr",
    "cer",
    "highlighted differences",
    "type of error",
    "Discrepancy Analysis"
]

for root, dirs, files in os.walk(GEMINI_JSON_DIR):
    for file in files:
        if not file.endswith(".json"):
            continue
        gemini_json_path = os.path.join(root, file)
        rel_path = os.path.relpath(gemini_json_path, GEMINI_JSON_DIR)
        human_json_path = os.path.join(HUMAN_JSON_DIR, rel_path)
        if not os.path.exists(human_json_path):
            print(f"❌ No matching human JSON for {gemini_json_path}")
            continue

        # Robust loading for Gemini JSON
        try:
            with open(gemini_json_path, "r", encoding="utf-8") as f:
                gemini_content = f.read()
                if not gemini_content.strip():
                    print(f"⚠️ Empty Gemini JSON: {gemini_json_path}")
                    continue
                gemini_data = json.loads(gemini_content)
        except Exception as e:
            print(f"❌ Error loading Gemini JSON: {gemini_json_path}\n{e}")
            continue

        # Robust loading for Human JSON
        try:
            with open(human_json_path, "r", encoding="utf-8") as f:
                human_content = f.read()
                if not human_content.strip():
                    print(f"⚠️ Empty Human JSON: {human_json_path}")
                    continue
                human_data = json.loads(human_content)
        except Exception as e:
            print(f"❌ Error loading Human JSON: {human_json_path}\n{e}")
            continue

        gemini_q = build_q_map(gemini_data)
        human_q = build_q_map(human_data)
        md_lines = ["# OCR Comparison Table", "\n", "| " + " | ".join(header) + " |", "| " + " | ".join(['---'] * len(header)) + " |"]
        cer_values = []
        for qnum in sorted(set(gemini_q.keys()) | set(human_q.keys())):
            human_text = sanitize(human_q.get(qnum, "na"))
            gemini_text = sanitize(gemini_q.get(qnum, "na"))
            if human_text == "na" or gemini_text == "na":
                cer_value = "na"
                highlighted_diff = "na"
            else:
                cer_value = char_error_rate(human_text, gemini_text)
                cer_values.append(cer_value)
                cer_value = f"{cer_value:.4f}"
                highlighted_diff = sanitize(highlight_differences(human_text, gemini_text))
            row = [
                file,
                str(qnum),
                human_text,
                gemini_text,
                cer_value,
                highlighted_diff,
                "",
                "",
            ]
            md_lines.append("| " + " | ".join(row) + " |")
        out_dir = os.path.join(OUTPUT_MD_DIR, os.path.dirname(rel_path))
        os.makedirs(out_dir, exist_ok=True)
        output_md_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.md")
        with open(output_md_path, "w", encoding="utf-8") as f:
            f.write("\n".join(md_lines))
        print(f"Markdown table written to {output_md_path}")
        # Write CER txt
        if cer_values:
            overall_cer = sum(cer_values) / len(cer_values)
            cer_txt_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.txt")
            with open(cer_txt_path, "w", encoding="utf-8") as f:
                f.write(f"Overall CER: {overall_cer:.4f}\n")
            print(f"CER written to {cer_txt_path}")
        else:
            print(f"No valid CER values for {file}")

❌ No matching human JSON for /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json/12_1002140198994121111692513661/12_1002140198994121111692513661.json
❌ No matching human JSON for /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json/09_1002114885961841111690700733/09_1002114885961841111690700733.json
❌ No matching human JSON for /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json/10_10021138351083421111694954514/10_10021138351083421111694954514.json
❌ No matching human JSON for /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Physics/Physics_Gemini/Physics_json/02_1002137635994121111692517447/02_10021376359941211116

In [ ]:
import os

md_path = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/maths_final.md"

def parse_md_table(md_lines):
    table = []
    for line in md_lines:
        line = line.strip()
        if line.startswith('|') and line.endswith('|'):
            cells = [cell.strip() for cell in line[1:-1].split('|')]
            table.append(cells)
    return table

with open(md_path, 'r') as f:
    lines = f.readlines()

table = parse_md_table(lines)

if len(table) < 2:
    print("Table too short or missing header.")
    exit()

header = table[0]
try:
    cer_idx = [h.lower().replace(' ', '') for h in header].index('cer')
except ValueError:
    print("No 'cer' column found in header.")
    exit()

cer_values = []
zero_cer_count = 0
for row in table[2:]:  # skip header and separator
    if cer_idx < len(row):
        val = row[cer_idx].strip().lower()
        if val != 'na' and val != '':
            try:
                cer_float = float(val)
                cer_values.append(cer_float)
                if cer_float == 0:
                    zero_cer_count += 1
            except ValueError:
                print(f"Non-numeric CER value ignored: {row[cer_idx]}")

if cer_values:
    avg_cer = sum(cer_values) / len(cer_values)
    print(f"Average CER (ignoring 'na'): {avg_cer}")
    print(f"Number of valid CER rows: {len(cer_values)}")
    print(f"Number of rows where CER is 0: {zero_cer_count}")
else:
    print("No valid CER values found.")

Non-numeric CER value ignored: BC AP/PB = AD/DC (B.P.T) → (1) Ad/DC = CE/BE (Given) → (2) AP/PB = CF/BE (From (1) & (2)) by converse of BPT DE//AC
Non-numeric CER value ignored: DE To prove AD/DB = AE/EC Join BE&#44; CD Construction : EF⊥AB &#44; DG⊥CA ∴ Area of AADE = 1/2 x base x haight = 1/2 x AD x EF = 1/2 x AE x DG Area of ΔPBE = Area of ΔADE / Area of ΔDBE = AD/DB → (1) Similarly Area of ΔADE = 1/2 x AE x DG Area of ΔDCE = 1/2 x EC x DG = AE/EC ΔDBE and ΔDCE have the same base b'w same parallels ∴ AD/DB = AC/EC
Non-numeric CER value ignored: gm bisect each other. So the midpoint of AC is same of mid point of BD AC BD (x₁+x₂/2&#44; y₁+y₂/2) = (3+x/2&#44; 6+y/2) (3+x/2&#44; 6+y/2) = (3+x/2&#44; 6+y/2) (6/2&#44; 8/2) = (3+x/2&#44; 6+y/2) 3 + x = 6 ; 6 + y = 3 x = 3 ; y = 2 So forth vertex's coordinates are (3&#44;2)
Non-numeric CER value ignored: \
Non-numeric CER value ignored: BC Given → AD/CD = AP/BP [BPT -(1)] Also given → AD/CD = CE/BE -(2) From (1) & (2) AD/CD = AP/BP = CE/BE 

In [64]:
import os
import json
from difflib import ndiff

GEMINI_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Maths_Gemini/Maths_json"
HUMAN_JSON_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Maths_human/pdf_docx_json"
OUTPUT_MD_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables"

os.makedirs(OUTPUT_MD_DIR, exist_ok=True)

def char_error_rate(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    insertions = sum(1 for d in diff if d[0] == '+')
    deletions = sum(1 for d in diff if d[0] == '-')
    ref_len = len(s1)
    if ref_len == 0:
        return 0 if len(s2) == 0 else 1
    cer = (insertions + deletions) / ref_len
    return cer

def highlight_differences(s1, s2):
    if s1 == "na" or s2 == "na":
        return "na"
    diff = list(ndiff(s1, s2))
    result = []
    for d in diff:
        if d[0] == ' ':
            result.append(d[2])
        elif d[0] == '-':
            result.append(f"[-{d[2]}-]")
        elif d[0] == '+':
            result.append(f"[+{d[2]}+]")
    return ''.join(result)

def sanitize(text):
    if text == "na":
        return text
    text = text.replace("\n", " ").replace("|", "\\|").replace(",", "&#44;")
    return text

def build_q_map(data):
    q_map = {}
    for entry in data:
        qnum = entry.get("question_number")
        text = entry.get("ocr_text", "")
        q_map[qnum] = text
    return q_map

header = [
    "docx source",
    "question number",
    "Human_ocr",
    "Gemini_ocr",
    "cer",
    "highlighted differences",
    "type of error",
    "Discrepancy Analysis"
]

for root, dirs, files in os.walk(GEMINI_JSON_DIR):
    for file in files:
        if not file.endswith(".json"):
            continue
        gemini_json_path = os.path.join(root, file)
        rel_path = os.path.relpath(gemini_json_path, GEMINI_JSON_DIR)
        human_json_path = os.path.join(HUMAN_JSON_DIR, rel_path)
        if not os.path.exists(human_json_path):
            print(f"❌ No matching human JSON for {gemini_json_path}")
            continue

        # Robust loading for Gemini JSON
        try:
            with open(gemini_json_path, "r", encoding="utf-8") as f:
                gemini_content = f.read()
                if not gemini_content.strip():
                    print(f"⚠️ Empty Gemini JSON: {gemini_json_path}")
                    continue
                gemini_data = json.loads(gemini_content)
        except Exception as e:
            print(f"❌ Error loading Gemini JSON: {gemini_json_path}\n{e}")
            continue

        # Robust loading for Human JSON
        try:
            with open(human_json_path, "r", encoding="utf-8") as f:
                human_content = f.read()
                if not human_content.strip():
                    print(f"⚠️ Empty Human JSON: {human_json_path}")
                    continue
                human_data = json.loads(human_content)
        except Exception as e:
            print(f"❌ Error loading Human JSON: {human_json_path}\n{e}")
            continue

        gemini_q = build_q_map(gemini_data)
        human_q = build_q_map(human_data)
        md_lines = ["# OCR Comparison Table", "\n", "| " + " | ".join(header) + " |", "| " + " | ".join(['---'] * len(header)) + " |"]
        cer_values = []
        for qnum in sorted(set(gemini_q.keys()) | set(human_q.keys())):
            human_text = sanitize(human_q.get(qnum, "na"))
            gemini_text = sanitize(gemini_q.get(qnum, "na"))
            if human_text == "na" or gemini_text == "na":
                cer_value = "na"
                highlighted_diff = "na"
            else:
                cer_value = char_error_rate(human_text, gemini_text)
                cer_values.append(cer_value)
                cer_value = f"{cer_value:.4f}"
                highlighted_diff = sanitize(highlight_differences(human_text, gemini_text))
            row = [
                file,
                str(qnum),
                human_text,
                gemini_text,
                cer_value,
                highlighted_diff,
                "",
                "",
            ]
            md_lines.append("| " + " | ".join(row) + " |")
        out_dir = os.path.join(OUTPUT_MD_DIR, os.path.dirname(rel_path))
        os.makedirs(out_dir, exist_ok=True)
        output_md_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.md")
        with open(output_md_path, "w", encoding="utf-8") as f:
            f.write("\n".join(md_lines))
        print(f"Markdown table written to {output_md_path}")
        # Write CER txt
        if cer_values:
            overall_cer = sum(cer_values) / len(cer_values)
            cer_txt_path = os.path.join(out_dir, f"{os.path.splitext(file)[0]}.txt")
            with open(cer_txt_path, "w", encoding="utf-8") as f:
                f.write(f"Overall CER: {overall_cer:.4f}\n")
            print(f"CER written to {cer_txt_path}")
        else:
            print(f"No valid CER values for {file}")

Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/08_10021013701065351171694350774/08_10021013701065351171694350774.md
CER written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/08_10021013701065351171694350774/08_10021013701065351171694350774.txt
Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/04_10021075241065351171694351737/04_10021075241065351171694351737.md
CER written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/04_10021075241065351171694351737/04_10021075241065351171694351737.txt
Markdown table written to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/soluti

In [76]:
import os
import re

# Paths
INPUT_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables"
OUTPUT_DIR = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Columns to extract
COLUMNS = ["Human_ocr", "Gemini_ocr", "type of error", "Discrepancy Analysis"]

# Function to correctly split the markdown row while handling escaped pipes
def split_md_row(row):
    # This regular expression splits the row by pipes while ignoring escaped pipes
    return re.split(r'(?<!\\)\|', row)[1:-1]  # Skip the first and last empty parts

# Iterate over all the subfolders in the input directory
for root, dirs, files in os.walk(INPUT_DIR):
    for file in files:
        if not file.endswith(".md"):
            continue
        
        input_md_path = os.path.join(root, file)
        output_md_path = os.path.join(OUTPUT_DIR, file)

        with open(input_md_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Find the header and the columns
        header_line = None
        for i, line in enumerate(lines):
            if line.strip().startswith("|"):
                header_line = i
                break

        if header_line is None:
            print(f"⚠️ No header row found in {input_md_path}")
            continue

        # Split the header and strip extra spaces from column names
        header = split_md_row(lines[header_line])
        header = [col.strip() for col in header]  # Strip spaces from column names
        print(f"Header found in {input_md_path}: {header}")  # Debugging line to inspect the header

        try:
            # Get the column indices for the desired columns
            col_indices = [header.index(col.strip()) for col in COLUMNS]  # Strip spaces in COLUMNS too
        except ValueError as e:
            print(f"⚠️ Could not find all columns in {input_md_path}: {e}")
            continue

        # Initialize output lines with the correct header and separator line
        out_lines = [
            "| " + " | ".join(COLUMNS) + " |",
            "| " + " | ".join(["---"] * len(COLUMNS)) + " |"
        ]
        
        # Iterate through the rows below the header
        for line in lines[header_line+2:]:
            if not line.strip().startswith("|"):
                continue
            row = split_md_row(line)  # Split the row
            if len(row) < len(header):
                continue  # Skip invalid rows

            # Extract only the desired columns
            new_row = [row[i].strip() for i in col_indices]
            out_lines.append("| " + " | ".join(new_row) + " |")

        # Write the processed table into a new markdown file
        with open(output_md_path, "w", encoding="utf-8") as f:
            f.write("\n".join(out_lines))

        print(f"Extracted columns to {output_md_path}")


Header found in /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/08_10021013701065351171694350774/08_10021013701065351171694350774.md: ['docx source', 'question number', 'Human_ocr', 'Gemini_ocr', 'cer', 'highlighted differences', 'type of error', 'Discrepancy Analysis']
Extracted columns to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md/08_10021013701065351171694350774.md
Header found in /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/Tables/04_10021075241065351171694351737/04_10021075241065351171694351737.md: ['docx source', 'question number', 'Human_ocr', 'Gemini_ocr', 'cer', 'highlighted differences', 'type of error', 'Discrepancy Analysis']
Extracted columns to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_gra

## send the 2 columns to gemini to get evalutaion 

In [80]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# === Load API Key ===
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_GEMINI_API"))

# Set model
model = genai.GenerativeModel("gemini-2.5-pro")

def send_md_and_prompt(input_md_path, prompt, output_md_dir):
    # Read Markdown file
    with open(input_md_path, 'r', encoding='utf-8') as f:
        md_content = f.read()

    # Compose prompt content
    full_prompt = f"{prompt}\n\n<Markdown Table Input>\n{md_content}"

    try:
        response = model.generate_content(
            full_prompt,
            generation_config={"temperature": 0.2},
        )
        generated_text = response.text

        # Try to extract Markdown from the response (strip markdown code block if present)
        if generated_text.strip().startswith('```markdown'):
            generated_text = generated_text.strip().removeprefix('```markdown').removesuffix('```').strip()
        elif generated_text.strip().startswith('```'):
            generated_text = generated_text.strip().removeprefix('```').removesuffix('```').strip()

        # Output file name
        base_name = os.path.splitext(os.path.basename(input_md_path))[0]
        os.makedirs(output_md_dir, exist_ok=True)
        output_md_path = os.path.join(output_md_dir, f"{base_name}_gemini_output.md")

        # Save the generated output
        with open(output_md_path, 'w', encoding='utf-8') as out_file:
            out_file.write(generated_text)

        print(f"🎉 Output saved to {output_md_path}")

    except Exception as e:
        print(f"❌ Error generating response: {e}")


# Function to process all .md files in a directory
def process_all_md_files(input_dir, output_dir, prompt):
    # Walk through all files in the directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".md"):
                input_md_path = os.path.join(root, file)
                output_md_dir = output_dir
                print(f"Processing {input_md_path}...")  # Log the current file being processed
                send_md_and_prompt(input_md_path, prompt, output_md_dir)


# === Example Usage ===
if __name__ == "__main__":
    input_md_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md"  # Directory containing .md files
    output_md_dir = "/Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/gemini_md"  # Directory for saving processed files
    prompt = """
Role: You are a highly accurate Quality Assurance (QA) Engine for OCR systems.
Context: You will be provided with a Markdown file containing two columns:
Human OCR: The text extracted by a human from the original content.
Gemini OCR: The text extracted by the Gemini model.
Objective: Your sole purpose is to determine which of the two columns—Human OCR or Gemini OCR—more accurately reflects the original content.
IMPORTANT ** : Make sure the content of the columns are present inside thier rrespective columns and rows

Specific Instructions:
1. For each row:
   - Compare the Human OCR and Gemini OCR for accuracy.
   - Determine the discrepancy (if any) between the two columns. Discrepancies can include:
     - Spelling mistakes
     - Punctuation errors
     - Wording differences
     - Numerical discrepancies
     - Missing Content
     - Extra Content
     - Content Mix-up
     - Omission

2. If either OCR text contains "na", mention "na" in the type of error and Discrepancy Analysis columns.

3. For each discrepancy found, you need to report it in a table with the following columns:
   - Question Number: The ID of the solution where the discrepancy occurred.
   - Human OCR: The full text of the solution from the Human OCR column.
   - Gemini OCR: The full text of the solution from the Gemini OCR column.
   - Type of error: The type of error (e.g., Spelling, Wording, Extra Content, Punctuation, Numerical Difference, Missing Content, Content Mix-up, Omission).
   - Discrepancy Analysis: A brief analysis of the error.

4. If no discrepancies are found, flag the Discrepancy Analysis as "NO ERRORS" and the Type of Error as "NO ERRORS".

Output: You will output a single table (in Markdown format) with discrepancies, including an analysis of the error type and a brief justification of which version is more accurate.
"""
    # Call the function to process all markdown files in the directory
    process_all_md_files(input_md_dir, output_md_dir, prompt)


Processing /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md/07_10021157131080491171694786726.md...
🎉 Output saved to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/gemini_md/07_10021157131080491171694786726_gemini_output.md
Processing /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md/04_10021075241065351171694351737.md...
🎉 Output saved to /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/gemini_md/04_10021075241065351171694351737_gemini_output.md
Processing /Users/simrannaik/Desktop/solution_improvement/ds-prototypes/subjective_grading/solution_improvement/Z_OCR_gemini_pcmb/Maths/analysis_md/01_10021165141080491171694788750.md...
🎉 Output saved to /Users/simranna

## to combine the tables of gemini and human ocr

In [ ]:
import os

# File paths
tables_md_path = "Z_OCR_gemini_pcmb/Maths/Tables/01_10021165141080491171694788750/01_10021165141080491171694788750.md"
gemini_md_path = "Z_OCR_gemini_pcmb/Maths/gemini_md/01_10021165141080491171694788750_gemini_output.md"
output_dir = "Z_OCR_gemini_pcmb/Maths/Final_MD"
output_md_path = os.path.join(output_dir, "01_10021165141080491171694788750_final.md")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

def parse_md_table(md_lines):
    """Parse markdown table lines into a list of lists (rows of columns)."""
    table = []
    for line in md_lines:
        line = line.strip()
        if line.startswith('|') and line.endswith('|'):
            # Split and strip each cell
            cells = [cell.strip() for cell in line[1:-1].split('|')]
            table.append(cells)
    return table

def format_md_table(table):
    """Format a list of lists back into markdown table string."""
    return '\n'.join(['| ' + ' | '.join(row) + ' |' for row in table])

# Read files
with open(tables_md_path, 'r') as f:
    tables_lines = f.readlines()
with open(gemini_md_path, 'r') as f:
    gemini_lines = f.readlines()

# Parse tables
tables_table = parse_md_table(tables_lines)
gemini_table = parse_md_table(gemini_lines)

# Sanity check: header and separator rows
header_rows = 2  # usually header + separator
tables_data = tables_table[header_rows:]
gemini_data = gemini_table[header_rows:]

# Remove last 2 columns from tables, get last 2 columns from gemini
final_table = []
# Add header and separator as is
final_table.extend(tables_table[:header_rows])

for t_row, g_row in zip(tables_data, gemini_data):
    trimmed = t_row[:-2]
    gemini_last2 = g_row[-2:]
    final_table.append(trimmed + gemini_last2)

# Write to output
with open(output_md_path, 'w') as f:
    f.write(format_md_table(final_table))

# Print row counts
print(f"Number of data rows in tables md: {len(tables_data)}")
print(f"Number of data rows in final md: {len(final_table) - header_rows}")
print(f"Final file saved to: {output_md_path}")